#### In this example we will look at how to use the K-Nearest_Neighbor algorithm for classification. We will use a modified version of the Video Store data set for this example. We will use the "Incidentals" attribute as the target attribute for classification (the class attribute). The goal is to be able to classify an unseen instance as "Yes" or "No" given the values of "Incidentals" from training instances.

In [1]:
import numpy as np
import pandas as pd

In [2]:
vstable = pd.read_csv("../../data/Video_Store_2.csv", index_col=0)

vstable.shape

(50, 7)

In [3]:
vstable.head()

,Gender,Income,Age,Rentals,Avg Per Visit,Genre,Incidentals
Cust ID,,,,,,,
1,M,45000,25,32,2.5,Action,Yes
2,F,54000,33,12,3.4,Drama,No
3,F,32000,20,42,1.6,Comedy,No
4,F,59000,70,16,4.2,Drama,Yes
5,M,37000,35,25,3.2,Action,Yes


#### We will be splitting the data into a test and training partions with the test partition to be used for evaluating model error-rate and the training partition to be used to find the K nearest neighbors. Before spliting the data we need to do a random reshuffling to make sure the instances are randomized.

In [4]:
 vs = vstable.reindex(np.random.permutation(vstable.index)) # we are shuffling the data first manually so that we have both classes both in the training and test data sets
vs.head(10) 

,Gender,Income,Age,Rentals,Avg Per Visit,Genre,Incidentals
Cust ID,,,,,,,
27,F,62000,47,32,3.6,Drama,No
24,F,79000,35,22,3.8,Drama,Yes
2,F,54000,33,12,3.4,Drama,No
26,F,56000,35,40,2.6,Action,Yes
21,F,47000,52,11,3.1,Drama,No
10,F,65000,40,21,3.3,Drama,No
13,M,83000,46,14,3.6,Comedy,No
36,F,29000,21,34,2.3,Comedy,No
47,F,69000,35,22,2.8,Drama,Yes


In [5]:
len(vs)

50

In [6]:
vs_names = vs.columns.values
vs_names # These are the names of the features that we are dealing with. 

array(['Gender', 'Income', 'Age', 'Rentals', 'Avg Per Visit', 'Genre',
       'Incidentals'], dtype=object)

##### Because this is classification we would like to separate our target from the rest of out matrix and we are putting it into vs_target

#### The target attribute for classification is Incidentals:

In [7]:
vs_target = vs.Incidentals
vs_target.head(10)
print(type(vs_target))

<class 'pandas.core.series.Series'>


#### Before we can compute distances we need to convert the data (excluding the target attribute containing the class labels) into standard spreadsheet format with binary dummy variables for categorical attributes).

In [8]:
# we exclude the incidentals, we can't have it here
# get dummies for everything that is not a number. 
vs = pd.get_dummies(vs[['Gender','Income','Age','Rentals','Avg Per Visit','Genre']])

In [9]:
type(vs)

pandas.core.frame.DataFrame

#### To be able to evaluate the accuracy of our predictions, we will split the data into training and test sets. In this case, we will use 80% for training and the remaining 20% for testing. Note that we must also do the same split to the target attribute.

In [10]:
# what we are trying to say, what is 80 percent of 50 (since the number of objects are 50 ) and the output will be the training
# the remaining 205 will be the testing data set.
tpercent = 0.8
tsize = int(np.floor(tpercent * len(vs)))
vs_train = vs[:tsize] # vs_train becomes up to tsize
vs_test = vs[tsize:] # vs_test becomes everything after tsize

In [11]:
print (vs_train.shape)
print (vs_test.shape)
# now we have two matrices
# we have the training and test  matrices
# (40 training items)
# (10 testing items)

(40, 9)
(10, 9)


In [12]:
np.set_printoptions(suppress=True, linewidth=120) # this is just I wanna things.

vs_train.head(10)

,Income,Age,Rentals,Avg Per Visit,Gender_F,Gender_M,Genre_Action,Genre_Comedy,Genre_Drama
Cust ID,,,,,,,,,
27,62000,47,32,3.6,1,0,0,0,1
24,79000,35,22,3.8,1,0,0,0,1
2,54000,33,12,3.4,1,0,0,0,1
26,56000,35,40,2.6,1,0,1,0,0
21,47000,52,11,3.1,1,0,0,0,1
10,65000,40,21,3.3,1,0,0,0,1
13,83000,46,14,3.6,0,1,0,1,0
36,29000,21,34,2.3,1,0,0,1,0
47,69000,35,22,2.8,1,0,0,0,1


In [13]:
vs_test # looking at the entire test data set since it is only ten items

,Income,Age,Rentals,Avg Per Visit,Gender_F,Gender_M,Genre_Action,Genre_Comedy,Genre_Drama
Cust ID,,,,,,,,,
3,32000,20,42,1.6,1,0,0,1,0
20,12000,16,23,2.2,0,1,1,0,0
4,59000,70,16,4.2,1,0,0,0,1
17,36000,35,28,3.5,0,1,0,0,1
9,38000,21,18,2.1,0,1,0,1,0
1,45000,25,32,2.5,0,1,1,0,0
11,41000,22,48,2.3,1,0,0,0,1
33,23000,25,28,2.7,0,1,1,0,0
15,68000,30,36,2.7,0,1,0,1,0


#### Splitting the target attribute ("Incidentals") accordingly:

In [14]:
# Method 1
vs_target_train = vs_target[:int(tsize)]
vs_target_test = vs_target[int(tsize):]
print(vs_target_test.shape)
vs_target_test

(10,)


Cust ID
3      No
20    Yes
4     Yes
17    Yes
9      No
1     Yes
11    Yes
33     No
15    Yes
22    Yes
Name: Incidentals, dtype: object

#### Whether you put the lower or upper limit like below it doesn't matter

In [15]:
# split the target attribute the same we split the data set into training and test.
# Method 2
vs_target_train = vs_target[0:int(tsize)]
vs_target_test = vs_target[int(tsize):len(vs)]
print(vs_target_test.shape)

(10,)


In [16]:
vs_target_train.head()

Cust ID
27     No
24    Yes
2      No
26    Yes
21     No
Name: Incidentals, dtype: object

In [17]:
vs_target_test

Cust ID
3      No
20    Yes
4     Yes
17    Yes
9      No
1     Yes
11    Yes
33     No
15    Yes
22    Yes
Name: Incidentals, dtype: object

#### Next, we normalize the attributes so that everything is in [0,1] scale. We can use the normalization functions from the kNN module in Ch. 2 of the text. In this case, however, we will use the more flexible and robust scaler function from the preprocessing module of scikit-learn package.

In [18]:
from sklearn import preprocessing

In [19]:
# we use the MinMaxScaler() from scikit-learn package
min_max_scaler = preprocessing.MinMaxScaler() 

# min_max scaling will be based on the training set and you apply that to the test set  (as we shall see below)
# so we assume that the min and the max that we know about is from the training set then when have a test set,
# we apply that range . This is tricky because we might have a test set that is some how not included in the original bounds
# so sometimes what people do, which is okay it is reseen the test data if you see it normalize. 
# so you can include it in your matrix, normalize the whole thing and  still not have seen the test data. 
# in this case we are assuming that the test set is going to nicely fit within our training set.  
min_max_scaler.fit(vs_train) 

MinMaxScaler(copy=True, feature_range=(0, 1))

In [20]:
vs_train_norm = min_max_scaler.fit_transform(vs_train) # apply it both on train and test
vs_test_norm = min_max_scaler.fit_transform(vs_test)
# if some reason you have a special case this won't work, it will complain and you will have to normalize your entire matrix
# can you please give an example?


#### Note that while these Scikit-learn functions accept Pandas dataframes as input, they return Numpy arrays (in this case the normalized versions of vs_train and vs_test).

In [21]:
np.set_printoptions(precision=2, linewidth=100)
vs_train_norm[:10]
# Everything has been normalized as you see below in the output. 

array([[0.69, 0.78, 0.59, 0.69, 1.  , 0.  , 0.  , 0.  , 1.  ],
       [0.89, 0.49, 0.33, 0.75, 1.  , 0.  , 0.  , 0.  , 1.  ],
       [0.6 , 0.44, 0.08, 0.64, 1.  , 0.  , 0.  , 0.  , 1.  ],
       [0.62, 0.49, 0.79, 0.42, 1.  , 0.  , 1.  , 0.  , 0.  ],
       [0.52, 0.9 , 0.05, 0.56, 1.  , 0.  , 0.  , 0.  , 1.  ],
       [0.73, 0.61, 0.31, 0.61, 1.  , 0.  , 0.  , 0.  , 1.  ],
       [0.93, 0.76, 0.13, 0.69, 0.  , 1.  , 0.  , 1.  , 0.  ],
       [0.32, 0.15, 0.64, 0.33, 1.  , 0.  , 0.  , 1.  , 0.  ],
       [0.77, 0.49, 0.33, 0.47, 1.  , 0.  , 0.  , 0.  , 1.  ],
       [0.55, 1.  , 0.15, 0.58, 1.  , 0.  , 0.  , 1.  , 0.  ]])

#### For consitency, we'll also convert the training and test target labels into Numpy arrays.

In [22]:
vs_target_train = np.array(vs_target_train)
vs_target_test = np.array(vs_target_test)

In [23]:
print (vs_target_train)
print ("\n")
print (vs_target_test)

['No' 'Yes' 'No' 'Yes' 'No' 'No' 'No' 'No' 'Yes' 'No' 'No' 'No' 'Yes' 'Yes' 'Yes' 'Yes' 'No' 'No'
 'Yes' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'No'
 'No' 'Yes' 'Yes' 'Yes']


['No' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'Yes' 'No' 'Yes' 'Yes']


#### The following function illustrates how we can perform a k-nearest-neighbor search. The "measure" argument allows us to use either Euclidean distance or (the inverse of) Cosine similarity as the distance function:

In [24]:
def knn_search(x, D, K, measure):
    """ find K nearest neighbors of an instance x among the instances in D """
    if measure == 0:
        # euclidean distances from the other points
        dists = np.sqrt(((D - x)**2).sum(axis=1))
    elif measure == 1:
        # first find the vector norm for each instance in D as wel as the norm for vector x
        D_norm = np.array([np.linalg.norm(D[i]) for i in range(len(D))])
        x_norm = np.linalg.norm(x) # cosine distance
        # Compute Cosine: divide the dot product o x and each instance in D by the product of the two norms
        sims = np.dot(D,x)/(D_norm * x_norm)
        # The distance measure will be the inverse of Cosine similarity
        dists = 1 - sims
    idx = np.argsort(dists) # sorting
    # return the indexes of K nearest neighbors
    return idx[:K], dists

In [25]:
# Let's use vs_test_norm[0] as a test instance x and find its K nearest neighbors
# the test matrix is ten items so I am taking the first feature vector
# I am giving my vs_train_norm data
# passing 5 neighbors
# passing 0 for Euclidean distance
neigh_idx, distances = knn_search(vs_test_norm[0], vs_train_norm, 5, 0)

In [26]:
vs_test_norm[0] # this is our first item that we passed

array([0.36, 0.07, 0.81, 0.  , 1.  , 0.  , 0.  , 1.  , 0.  ])

In [27]:
print (neigh_idx) # after running it through KNN search and it is going to tell us the closest documents to the query object
print ("\n")
vs_train.iloc[neigh_idx]  # when put the indicies inside the training set, we know what these documents were. 
# so these are the 5 closest neighbors

[ 7 36 29 17  9]




,Income,Age,Rentals,Avg Per Visit,Gender_F,Gender_M,Genre_Action,Genre_Comedy,Genre_Drama
Cust ID,,,,,,,,,
36,29000,21,34,2.3,1,0,0,1,0
19,24000,25,41,3.1,1,0,0,1,0
23,2000,15,30,2.5,1,0,0,1,0
39,68000,35,19,3.9,1,0,0,1,0
31,49000,56,15,3.2,1,0,0,1,0


In [28]:
print (distances[neigh_idx]) # we look at the distances

[0.38 0.59 0.59 1.12 1.29]


In [29]:
# so  we put these indicies inside our training label and findout what our labels were
neigh_labels = vs_target_train[neigh_idx] 
print (neigh_labels)

['No' 'No' 'No' 'No' 'No']


#### Now that we know the nearest neighbors, we need to find the majority class label among them. The majority class would be the class assgined to the new instance x.

In [30]:
from collections import Counter
print (Counter(neigh_labels)) # count the occurences of the unique values in line 66

Counter({'No': 5})


In [31]:
# count the occurences of the unique values in line 66
# and the look at the most and that is your vote.
Counter(neigh_labels).most_common(1) # This is your vote

[('No', 5)]

#### Next, we'll use the Knn module from Chapter 2 of Machine Learning in Action. Before importing the whole module, let's illustrate what the code does by stepping through it with some specific input values.

In [32]:
dataSetSize = vs_train_norm.shape[0]
print (dataSetSize) # this is normalized training matrix shape
# print(vs_train_norm.shape)

40


In [33]:
# inX = vs_test_norm[0]
# print(inX)
# print(inX.shape) #(9,)
# test = np.tile(inX, (dataSetSize, 1)) # search for tile
# #print(test)
# print(test.shape) # (40,9)

In [34]:
inX = vs_test_norm[0]   # We'll use the first instance in the test data for this example
diffMat = np.tile(inX, (dataSetSize,1)) - vs_train_norm  # Create dataSetSize copies of inX, as rows of a 2D matrix
                                                         # Compute a matrix of differences
                                                        # create a copy of the queries in the size of the training matrix shape
                                                        # subtracting norm from every single copy
print (diffMat[:5,:]) # so this is the difference matrix

[[-0.34 -0.71  0.22 -0.69  0.    0.    0.    1.   -1.  ]
 [-0.53 -0.41  0.48 -0.75  0.    0.    0.    1.   -1.  ]
 [-0.25 -0.36  0.74 -0.64  0.    0.    0.    1.   -1.  ]
 [-0.27 -0.41  0.02 -0.42  0.    0.   -1.    1.    0.  ]
 [-0.17 -0.83  0.76 -0.56  0.    0.    0.    1.   -1.  ]]


In [35]:
# This is just the long way of doing the Euclidean distance.
# you have to square those differences.
sqDiffMat = diffMat**2  # The matrix of squared differences

# you take the sum of the squared differences
sqDistances = sqDiffMat.sum(axis=1)  # 1D array of the sum of squared differences (one element for each training instance)

# you take the sqaure root of that, which is Euclidean distance. 
distances = sqDistances**0.5  # and finally the matrix of Euclidean distances to inX

# this is basically doing the whole operation we were doing earlier but this is step by step with entire matrix
print (distances)

[1.77 1.8  1.77 1.55 1.9  1.75 1.94 0.38 1.67 1.29 2.04 1.82 1.64 2.2  2.1  2.16 1.76 1.12 2.11
 2.06 1.46 1.46 2.14 2.07 2.25 1.92 2.03 2.16 2.21 0.59 2.06 1.52 1.85 1.52 1.74 1.6  0.59 2.29
 1.58 2.11]


In [36]:
# we sort the distances using the argsort() function
sortedDistIndicies = distances.argsort() # the indices of the training instances in increasing order of distance to inX

# print the sorted indicies
print (sortedDistIndicies)
# so we have the indicies of the ordered distance 

[ 7 36 29 17  9 20 21 31 33  3 38 35 12  8 34  5 16  2  0  1 11 32  4 25  6 26 10 19 30 23 14 18 39
 22 27 15 13 28 24 37]


#### To see how the test instance should be classified, we need to find the majority class among the neighbors (here we do not use distance weighting; only a simply voting approach)

In [37]:
classCount={}
k = 5  # We'll use the top 5 neighbors
for i in range(k):
    # find the location of the sorted distances from vs_tartget_train
   voteIlabel = vs_target_train[sortedDistIndicies[i]]
    #print('voteIlabel', voteIlabel)
    
    # count the class of the closest neighbors
    # classCount gets the label that you have above ( in voteIlabel) which you keep incrementing whenever you find a yes
    # you increment a No if it is a No
   classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1  # add to the count of the label or retun 1 for first occurrence
   print (sortedDistIndicies[i], voteIlabel, classCount[voteIlabel])


7 No 1
36 No 2
29 No 3
17 No 4
9 No 5


#### Now, let's find the predicted class for the test instance.

In [38]:
import operator
# Create a dictionary for the class labels with cumulative occurrences across the neighbors as values
# Dictionary will be ordered in decreasing order of the lable values (so the majority class label will
# be the first dictonary element)
sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
print (sortedClassCount)
print (sortedClassCount[0][0])

[('No', 5)]
No


#### A better way to find the majority class given a list of class labels from neighbors:

In [39]:
from collections import Counter

k = 5  # We'll use the top 5 neighbors
vote = vs_target_train[sortedDistIndicies[0:k]]
maj_class = Counter(vote).most_common(1)

print (vote)

print (maj_class)

print ("Class label for the classified istance: ", maj_class[0][0])

['No' 'No' 'No' 'No' 'No']
[('No', 5)]
Class label for the classified istance:  No


#### Let's know import the whole KNN module from MLA (uploaded on D2L) and use as part of a more robust evaluation process. We will step through all test instances, use our Knn classifier to predict a class label for each instance, and in each case we compare the predicted label to the actual value from the target test labels.

In [40]:
import kNN # importing KNN.py from MLA

In [41]:
numTestVecs = len(vs_target_test) # how many test items we have 
print (numTestVecs) # we are just looking at how many test items we have 

10


In [42]:
errorCount = 0.0
for i in range(numTestVecs):
    # classify0 function uses Euclidean distance to find k nearest neighbors
    # vs_test_norm[i, :] is your query
    # vs_train_norm is your training data
    # vs_target_train is your target from the training data so that you can know your votes
    # 3 is nearest neighbor k = 3
    classifierResult = kNN.classify0(vs_test_norm[i,:], vs_train_norm, vs_target_train, 3)
    print ("the classifier came back with: %s, the real answer is: %s" % (classifierResult, vs_target_test[i]))
    # if is is not the same then you ad your errors or you increment your errors
    # I classify something and I say it is a Yes, but the actual was No so that is an error
    if (classifierResult != vs_target_test[i]): errorCount += 1.0
  # so you can just count the number of mistakes you made      
print ("the total error rate is: %f" % (errorCount/float(numTestVecs)))
# it has 40% errors

the classifier came back with: No, the real answer is: No
the classifier came back with: No, the real answer is: Yes
the classifier came back with: No, the real answer is: Yes
the classifier came back with: Yes, the real answer is: Yes
the classifier came back with: No, the real answer is: No
the classifier came back with: Yes, the real answer is: Yes
the classifier came back with: Yes, the real answer is: Yes
the classifier came back with: Yes, the real answer is: No
the classifier came back with: No, the real answer is: Yes
the classifier came back with: Yes, the real answer is: Yes
the total error rate is: 0.400000


#### I have added a new classifier function to the kNN module that uses Cosine similarity instead of Euclidean distance:

In [43]:
from importlib import reload
reload(kNN)

<module 'kNN' from 'C:\\Users\\rejalu1\\OneDrive - Henry Ford Health System\\DSC478MachineLearning\\Notebooks\\week3\\kNN.py'>

In [44]:
errorCount = 0.0
for i in range(numTestVecs):
    # classify1 function uses inverse of Cosine similarity to find k nearest neighbors
    classifierResult2 = kNN.classify1(vs_test_norm[i,:], vs_train_norm, vs_target_train, 3) # doing cosine
    print ("the classifier came back with: %s, the real answer is: %s" % (classifierResult2, vs_target_test[i]))
    if (classifierResult2 != vs_target_test[i]): errorCount += 1.0
print ("the total error rate is: %f" % (errorCount/float(numTestVecs)))

the classifier came back with: No, the real answer is: No
the classifier came back with: No, the real answer is: Yes
the classifier came back with: No, the real answer is: Yes
the classifier came back with: Yes, the real answer is: Yes
the classifier came back with: No, the real answer is: No
the classifier came back with: Yes, the real answer is: Yes
the classifier came back with: Yes, the real answer is: Yes
the classifier came back with: Yes, the real answer is: No
the classifier came back with: No, the real answer is: Yes
the classifier came back with: Yes, the real answer is: Yes
the total error rate is: 0.400000
